In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import pandas as pd

In [25]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

In [26]:
# Load pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [27]:
# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Add a new pad token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [28]:
data = pd.read_csv('drive/My Drive/AI Final Project/general_knowledge_qa.csv')

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   question       930 non-null    object
 1   answer         930 non-null    object
 2   question_type  930 non-null    object
 3   image          11 non-null     object
dtypes: object(4)
memory usage: 29.2+ KB


In [30]:
data.drop('image', axis= 1, inplace= True)

In [31]:
data.head()

,question,answer,question_type
0,How many days do we have in a week?,Seven,General Knowledge For Kids
1,How many days are there in a normal year?,365 (not a leap year),General Knowledge For Kids
2,How many colors are there in a rainbow?,7,General Knowledge For Kids
3,Which animal is known as the ‘Ship of the Dese...,Camel,General Knowledge For Kids
4,How many letters are there in the English alph...,26,General Knowledge For Kids


In [32]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

In [33]:
#Cleaning data

# Use str.strip() on the 'question' column
data['question'] = data['question'].str.strip()

# Clean the 'question_type' column
data['question_type'] = data['question_type'].str.strip().str.lower()


In [34]:

questions = data['question']
question_types = data['question_type']

In [35]:
# Create an empty list to store generated questions
generated_questions = []

In [36]:
# Generate questions for each unique question type
max_length = 50
num_beams = 5
no_repeat_ngram_size = 2

for question_type in data['question_type'].unique():
    # Encode the question type
    input_ids = tokenizer.encode(question_type, return_tensors='pt', padding=True, truncation=True)

    # Generate a question for the current question type
    output_ids = model.generate(input_ids, max_length=max_length, num_beams=num_beams, no_repeat_ngram_size=no_repeat_ngram_size)
    generated_question = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Append the generated question to the list
    generated_questions.append(generated_question)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

In [37]:
# Create a DataFrame to store the results
generated_data = pd.DataFrame({
    'question_type': data['question_type'].unique(),
    'generated_question': generated_questions
})

In [38]:
# Print or explore generated questions
print(generated_data[['question_type', 'generated_question']])

                question_type  \
0  general knowledge for kids   
1    gk questions for class 1   
2    gk questions for class 2   
3    gk questions for class 3   
4    gk questions for class 4   
5    gk questions for class 5   
6    gk questions for class 6   
7    gk questions for class 7   

                                  generated_question  
0  general knowledge for kids.\n\n"I think it's r...  
1  gk questions for class 1, 2, 3, 4, 5, 6, 7, 8,...  
2  gk questions for class 2, 3, 4, 5, 6, 7, 8, 9,...  
3  gk questions for class 3, 4, 5, 6, 7, 8, 9, 10...  
4  gk questions for class 4 students.\n\n"I'm not...  
5  gk questions for class 5 students.\n\nThe clas...  
6  gk questions for class 6 students.\n\nThe clas...  
7                 gk questions for class 7 students.  


Evaluation

In [39]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

In [40]:
# Sample reference and candidate questions
reference_question = "How many days in a week?"
candidate_question = "How many days are there in one week?"

# Tokenize the sentences
reference_tokens = nltk.word_tokenize(reference_question.lower())
candidate_tokens = nltk.word_tokenize(candidate_question.lower())

In [41]:
# BLEU Score
bleu_score = sentence_bleu([reference_tokens], candidate_tokens)
print(f"BLEU Score: {bleu_score}")

BLEU Score: 5.30941411456236e-78


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Saving Model

In [45]:
# Save model weights and configuration to a specific directory
save_path = 'drive/My Drive/AI Final Project/saved_model'
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('drive/My Drive/AI Final Project/saved_model/tokenizer_config.json',
 'drive/My Drive/AI Final Project/saved_model/special_tokens_map.json',
 'drive/My Drive/AI Final Project/saved_model/vocab.json',
 'drive/My Drive/AI Final Project/saved_model/merges.txt',
 'drive/My Drive/AI Final Project/saved_model/added_tokens.json')